In [ ]:
import os
import json
import base64
from google.oauth2.service_account import Credentials
from oauth2client.service_account import ServiceAccountCredentials
import gspread


In [38]:
# Decode the Google Service Account from environment variable
GOOGLE_SA = os.environ.get("GOOGLE_SERVICE_ACCOUNT")
sa_decoded = base64.b64decode(GOOGLE_SA, validate=True)
sa_json = json.loads(sa_decoded)

# Google Sheets API scope
SCOPE = [
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive",
]

# Authenticate and create the gspread client
gc = gspread.service_account_from_dict(sa_json, scopes=SCOPE)
sheet_id = os.environ["GOOGLE_SHEET_ID_MATCH_SUMMARY"]

In [ ]:
sh = gc.open_by_key(sheet_id)
print(sh)

In [ ]:
import requests
import os
import pandas as pd
from sqlalchemy import create_engine, text

In [ ]:
API_URL = "https://padelapi.org/api/players/"
API_TOKEN = os.environ["PADEL_API_TOKEN"]

headers = {
    "Authorization": f"Bearer {API_TOKEN}",
    "Accept": "application/json"
}

params = {
    "limit": 100,
    "offset": 0
}

In [ ]:
response = requests.get(API_URL, headers=headers)

In [ ]:
json_data = response.json()["data"]
df_players = pd.json_normalize(json_data)
df_players.head()

In [ ]:
# Step 1: Connect to PostgreSQL
username = os.environ["POSTGRES_USER"]
password = os.environ["POSTGRES_PWD"]
host = os.environ["POSTGRES_HOST"]
database = os.environ["POSTGRES_DB"]
engine = create_engine(f"postgresql+psycopg2://{username}:{password}@{host}/{database}")

# Step 2: Load the DataFrame into PostgreSQL
table_name = "players"
df_players.to_sql(table_name, engine, if_exists="replace", index=False)

print(f"✅ Data successfully loaded into table '{table_name}' in database '{database}'.")

In [ ]:
import os

# Postgres configuration
username = os.environ["POSTGRES_USER"]
password = os.environ["POSTGRES_PWD"]
host = os.environ["POSTGRES_HOST"]
database = os.environ["POSTGRES_DB"]

engine_url = f"postgresql+psycopg2://{username}:{password}@{host}/{database}"
engine = create_engine(engine_url, pool_pre_ping=True)

In [ ]:
# Read from Postgres DB
with engine.connect() as connection:
    df_matches = pd.read_sql_query(text("SELECT * FROM matches"), connection)
    
engine.dispose()
print(len(df_matches))
df_matches.head()

In [ ]:
df_matches[df_matches["duration"].isnull()]

In [ ]:
df_matches["duration_minutes"] = (
    df_matches["duration"]
      .dropna()
      .str.split(":", expand=True)
      .astype(int)
      .pipe(lambda x: x[0] * 60 + x[1])
)


In [ ]:
df_matches[["duration_minutes", "duration"]]

In [ ]:

df_summary = pd.DataFrame()
df_summary = df_matches \
    .groupby(['category','played_at']) \
    .agg(
        match_count=('played_at', 'count'),
        avg_duration_minutes=('duration', 'mean'),
        last_played_at=('played_at', 'max')
    ).size().reset_index(name='match_count')

Dimensions: by category and day

Measures: count of matches, avg duration (minutes), #days since last match (d)